In [6]:
import os
import time
import sys

# Related major packages
import anuga

# Application specific imports              # Definition of file names and polygons
import aw_small_project

from numpy import allclose
import numpy as np
from math import sin
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import csv
from anuga.utilities.numerical_tools import ensure_numeric
from anuga.geospatial_data.geospatial_data import ensure_absolute

from csv import reader,writer
from anuga.utilities.numerical_tools import ensure_numeric, mean, NAN
import string
from anuga.utilities.file_utils import get_all_swwfiles
from anuga.abstract_2d_finite_volumes.util import file_function  
from anuga.geospatial_data.geospatial_data import ensure_absolute
import gc
from csv import reader,writer
from anuga.utilities.numerical_tools import ensure_numeric, mean, NAN
import string
from anuga.utilities.file_utils import get_all_swwfiles
from anuga.abstract_2d_finite_volumes.util import file_function  
from anuga.geospatial_data.geospatial_data import ensure_absolute
from anuga.geospatial_data.geospatial_data import ensure_absolute
from anuga.file.netcdf import NetCDFFile
from anuga.config import netcdf_mode_r, netcdf_mode_w, netcdf_mode_a
from anuga.utilities.numerical_tools import ensure_numeric
from anuga.fit_interpolate.interpolate import Interpolate
from anuga.abstract_2d_finite_volumes.neighbour_mesh import Mesh
from anuga.geometry.polygon import outside_polygon

from func_tools import get_matrix_A, read_result, input_matrix

Area of bounding polygon = 455.06 km^2


In [8]:
# First have a sample sww to show the vertice and other staff
# The Simulation 
# saving file place
saving_file = 'E:/austinflood_sww/flood_result/'
sww_dir = 'E:/austinflood_sww'
sww_name = 'aw_small_simu_3'
sww_fn = sww_dir + '/' + sww_name + '.sww'
gauge_fn = 'GISdata&mesh/aw_small_gauges_100.csv'

quantities = ['stage', 'xmomentum', 'ymomentum']
# The Data needed
# suppose the whole area is 400 km^2
# the rain range is from 1mm/hour to 10mm/hour to 50 mm/hour to 100mm/hour
# correspondingly, the M^3/s is 111,1111,5555.5,11111 m3/s
num_case = 6
num_grids = 9.0
H = [111.0, 1111.0, 5555.0, 11111.0, 22222.0]
H = [ele/num_grids/8.0*9.0 for ele in H]
num_h = len(H) - 1
St = [1*60.0, 30*60.0]
num_st = len(St) - 1
Dt = [30*60.0, 360*60.0]
num_dt = len(Dt) - 1
A = [0.5, 3.0, 10.0]
num_a = len(A) - 1
FRICTION = [0.04, 0.08]
num_friction = len(FRICTION) - 1
BH = [155.0, 160.0]
BL = [115.0, 120.0]
num_boundary = len(BH) - 1
INFLOW_MAIN = [20, 100, 500, 1500]
# INFLOW_DAM = [], 80 percent the inflow
INFLOW_SMALL = [10, 100, 300, 500]
num_inflow = len(INFLOW_MAIN) - 1

o_xmomentum = 0
o_ymomentum = 0

time00 = time.time()
rain_begin_time = 120*60
# Start to load the GIS data for the mesh
# Create DEM from asc data
anuga.asc2dem('GISdata&mesh/rpaw_clip.asc', use_cache=aw_small_project.cache, 
              verbose=False)
# Create pts file for onshore DEM
anuga.dem2pts('GISdata&mesh/rpaw_clip.dem', use_cache=aw_small_project.cache, 
              verbose=False)
domain = anuga.create_domain_from_regions(aw_small_project.bounding_polygon,
                                    boundary_tags={'top': [0],
                                                   'right': [1],
                                                   'bottom': [2],
                                                   'left': [3]},
                                    maximum_triangle_area=aw_small_project.background_res,
                                    mesh_filename=aw_small_project.meshname,
                                    use_cache=aw_small_project.cache,
                                    verbose=False)

# Print some stats about mesh and domain
# print 'Number of triangles = ', len(domain)
# print 'The extent is ', domain.get_extent()
# print domain.statistics()
domain.set_name(sww_name) # Name of sww file
domain.set_datadir(sww_dir)    # Store sww output here
domain.set_minimum_storable_height(0.01)      # Store only depth > 1cm
domain.set_flow_algorithm('DE0')
domain.set_quantity('elevation', 
                    filename='GISdata&mesh/rpaw_clip.pts',
                    use_cache=aw_small_project.cache,
                    verbose=False,
                    alpha=0.1)

# read the polygon of rain area
rain_area0_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_0.csv')
rain_area1_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_1.csv')
rain_area2_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_2.csv')
rain_area3_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_3.csv')
rain_area4_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_4.csv')
rain_area5_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_5.csv')
rain_area6_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_6.csv')
rain_area7_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_7.csv')
rain_area8_polygon = anuga.read_polygon('GISdata&mesh/rain_polygon_8.csv')

# get the matrix_a and elevation 
mark = 0
for case in range(2, 3):
    for i_st in range(num_st):
#for i_st in range(1):
        for i_dt in range(num_dt):
    #for i_dt in range(1):
            for i_a in range(num_a):
        #for i_a in range(1):
                for i_boundary in range(num_boundary):
            #for i_boundary in range(1):
                    for i_flow in range(num_inflow):
                #for i_flow in range(1):
                        for ih in range(num_h):
                    #for ih in range(1):
                            for i_friction in range(num_friction):
                        #for i_friction in range(1):
                                input_info = []
                                case_name = '{0}_{1}_{2}_{3}_{4}_{5}_{6}_{7}'.format(i_st, i_dt, i_a, i_boundary,
                                                                                     i_flow,ih,i_friction,case)
                                # set the time
                                domain.finaltime = None
                                domain.recorded_min_timestep = domain.recorded_max_timestep = 0.0
                                domain.starttime = 0 
                                domain.evolve_starttime = 0.0
                                domain.time = 0.0
                                domain.timestep = 0.0
                                domain.flux_timestep = 0.0
                                domain.evolved_called = False
                                # First delete previous status stored
                                # set the xmomentum and ymomentum to be the river's speed approximation
                                # the meaning of xmomentum is uh speed*height
                                print('This is case '+case_name)
                                domain.set_quantity('xmomentum', o_xmomentum)
                                domain.set_quantity('ymomentum', o_ymomentum)
                                # delete the rain inputs
                                length_force = len(domain.forcing_terms)
                                del domain.forcing_terms[0:length_force]
    
                                friction_coef = FRICTION[i_friction] + (FRICTION[i_friction+1] - FRICTION[i_friction])*np.random.rand(1)
                                #print(friction_coef)
                                #stage_height = 150.0
                                # set the new coefficients of the domain
                                domain.set_quantity('friction', np.float(friction_coef))
                                #domain.set_quantity('stage', stage_height)
                                init_h = 0.0
                                domain.set_quantity('stage',expression = 'elevation + %f' % init_h)
    
                                time01 = time.time()
                                # print 'That took %.2f seconds to fit data' %(time01-time00)
                                if aw_small_project.just_fitting:
                                    import sys
                                    sys.exit()
                                # print 'Available boundary tags', domain.get_boundary_tags()
    
                                # boundary conditions(needs to be fixed here)
                                Bh = BH[i_boundary] + (BH[i_boundary+1] 
                                                       - BH[i_boundary])*np.random.rand(1)
                                Bl = BL[i_boundary] + (BL[i_boundary+1] 
                                                       - BL[i_boundary])*np.random.rand(1)
                                Bh = anuga.Dirichlet_boundary([np.float(Bh), 0, 0]) # Mean water level
                                Bl = anuga.Dirichlet_boundary([np.float(Bl), 0, 0]) # Neutral boundary

                                domain.set_boundary({'top': Bl,
                                                     'right': Bl,
                                                     'left': Bh,
                                                     'bottom': Bl})
    
                                # inflow of river
                                # first we need to fulfill the water with water
                                # parameters for the river inflow 890cfs = 25m^3/s
    
                                #inflow1 = anuga.Inflow(domain, rate =lambda t: 30 + 10*sin(t/600), center=(613697.5, 3356877.7), radius=80)
                                #inflow1_1 = anuga.Inflow(domain, rate =lambda t: 20000 + 3000*sin(t/10), center=(616696.700,3351594.169), radius=80)
                                inflow1_1 = anuga.Inflow(domain, rate =lambda t: 20000 , center=(614402.9,3357988.169),
                                                         radius=80)
                                domain.forcing_terms.append(inflow1_1)
                                inflow2_1 = anuga.Inflow(domain, rate =lambda t: 5000 , center=(616469.9,3359135.169), 
                                                         radius=120)
                                domain.forcing_terms.append(inflow2_1)
                                inflow3_1 = anuga.Inflow(domain, rate =lambda t: 5000, center=(615534.9,3353404.169), 
                                                         radius=120)
                                domain.forcing_terms.append(inflow3_1)
                                inflow4_1 = anuga.Inflow(domain, rate =lambda t: 8000 , center=(616555.9,3347799.169), 
                                                         radius=120)
                                domain.forcing_terms.append(inflow4_1)
                                inflow5_1 = anuga.Inflow(domain, rate =lambda t: 4000, center=(631876.9,3340564.169), 
                                                         radius=120)
                                domain.forcing_terms.append(inflow5_1)
                                inflow6_1 = anuga.Inflow(domain, rate =lambda t: 30000 , center=(630315.9,3348766.169),
                                                         radius=120)
                                domain.forcing_terms.append(inflow6_1)
                                inflow7_1 = anuga.Inflow(domain, rate =lambda t: 7000 , center=(616779.9,3351179.169),
                                                         radius=120)
                                domain.forcing_terms.append(inflow7_1)
                                inflow8_1 = anuga.Inflow(domain, rate =lambda t: 7000 , center=(618567.9,3349389.169),
                                                         radius=120)
                                domain.forcing_terms.append(inflow8_1)
                                inflow9_1 = anuga.Inflow(domain, rate =lambda t: 7000 , center=(621919.9,3347113.169),
                                                         radius=180)
                                domain.forcing_terms.append(inflow9_1)
                                inflow10_1 = anuga.Inflow(domain, rate =lambda t: 7000 , center=(616719.9,3351941.169),
                                                          radius=120)
                                domain.forcing_terms.append(inflow10_1)
                                
                                length_fulfill_force = len(domain.forcing_terms)
    
                                # inflow parameters
                                in1 = INFLOW_MAIN[i_flow] + (INFLOW_MAIN[i_flow+1] 
                                                             - INFLOW_MAIN[i_flow])*np.random.rand(1)
                                in1 = np.float(in1)
                                in2 = INFLOW_SMALL[i_flow] + (INFLOW_SMALL[i_flow+1] 
                                                             - INFLOW_SMALL[i_flow])*np.random.rand(1)
                                in2 = np.float(in2)
                                in3 = INFLOW_SMALL[i_flow] + (INFLOW_SMALL[i_flow+1] 
                                                             - INFLOW_SMALL[i_flow])*np.random.rand(1)
                                in3 = np.float(in3)
                                in4 = 1.5*INFLOW_SMALL[i_flow] + 1.5*(INFLOW_SMALL[i_flow+1] 
                                                             - INFLOW_SMALL[i_flow])*np.random.rand(1)
                                in4 = np.float(in4)
                                in5 = INFLOW_SMALL[i_flow] + (INFLOW_SMALL[i_flow+1] 
                                                             - INFLOW_SMALL[i_flow])*np.random.rand(1)
                                in5 = np.float(in5)
                                in10 = INFLOW_MAIN[i_flow] + (INFLOW_MAIN[i_flow+1] 
                                                              - INFLOW_MAIN[i_flow])*np.random.rand(1)
                                in10 = np.float(in10)
                                out1 = -INFLOW_MAIN[i_flow] - (INFLOW_MAIN[i_flow+1]
                                                              - INFLOW_MAIN[i_flow])*np.random.rand(1)
                                out1 = np.float(out1)
    
                                inflow1_2 = anuga.Inflow(domain, rate =lambda t: in1 + 0.1*in1*sin(t/10.0),
                                                         center=(614402.9,3357988.169), radius=80)
                                inflow2_2 = anuga.Inflow(domain, rate =lambda t: in2 + 0.1*in2*sin(t/10.0), 
                                                         center=(616469.9,3359135.169), radius=120)
                                inflow3_2 = anuga.Inflow(domain, rate =lambda t: in3 + 0.1*in3*sin(t/10.0), 
                                                         center=(615534.9,3353404.169), radius=120)
                                inflow4_2 = anuga.Inflow(domain, rate =lambda t: in4 + 0.1*in4*sin(t/10.0), 
                                                         center=(616555.9,3347799.169), radius=120)
                                inflow5_2 = anuga.Inflow(domain, rate =lambda t: in5 + 0.1*in5*sin(t/10.0), 
                                                         center=(631876.9,3340564.169), radius=120)
                                #inflow6_2 = anuga.Inflow(domain, rate =lambda t: in6 + 3*sin(t/10), center=(630315.9,3348766.169), radius=120)
                                inflow10_2 = anuga.Inflow(domain, rate =lambda t: in10 + 0.1*in10*sin(t/10.0) , 
                                                          center=(616719.9,3351941.169), radius=120.0)
                                outflow1 = anuga.Inflow(domain, rate =lambda t: out1 - 0.1*out1*sin(t/10.0) , 
                                                        center=(616719.9,3351941.169), radius=120)
                                                               
                                domain.forcing_terms.append(inflow1_2)
                                domain.forcing_terms.append(inflow2_2)
                                domain.forcing_terms.append(inflow3_2)
                                domain.forcing_terms.append(inflow4_2)
                                domain.forcing_terms.append(inflow5_2)
                                domain.forcing_terms.append(inflow10_2)
                                domain.forcing_terms.append(outflow1)

                                #xmom_1 = anuga.shallow_water.forcing.General_forcing(domain, 'xmomentum', rate =5.0, center=(616696.700,3351594.169), radius=80) 
                                #domain.forcing_terms.append(xmom_1)
                                #ymom_1 = anuga.shallow_water.forcing.General_forcing(domain, 'ymomentum', rate =-3.0, center=(616696.700,3351594.169), radius=80) 
                                #domain.forcing_terms.append(ymom_1)
    
                                # inflow2 = anuga.Inflow(domain, rate = -10 ,center=(672749.0- 598540, 3336967.0- 3328630), radius=20)
                                # inflow2 = anuga.Inflow(domain, rate =lambda t: -25 + 10*sin(t/10) ,center=(634506.0, 3343667.0), radius=200)
                                # domain.forcing_terms.append(inflow2)
    
                                # inflow of rain
                                # st: time before starting to rain(seconds)
                                # dt: the time to reach peak(seconds)
                                # h: the peak value (m^3/s)
                                # a: the coef to the exponential function
                                st1 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt1 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h1 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a1 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b1= (st1/60/60.0+dt1/60/60.0) + np.log(h1)/a1
                                rain_inflow_0 = anuga.Inflow(domain,  polygon = rain_area0_polygon, 
                                                             rate = lambda t: max(min(h1/(dt1/60/60.0)
                                                                                      *(((t-rain_begin_time)/(60*60.0))
                                                                                        - st1/60/60.0)
                                                                                      ,np.exp(-a1*((t-rain_begin_time)
                                                                                                   /(60*60.0)-b1))),0))
                                st2 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt2 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h2 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a2 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b2= (st2/60/60.0+dt2/60/60.0) + np.log(h2)/a2
                                rain_inflow_1 = anuga.Inflow(domain, polygon = rain_area1_polygon, 
                                                             rate = lambda t: max(min(h2/(dt2/60/60.0)
                                                                                      *(((t-rain_begin_time)/(60*60.0))
                                                                                        - st2/60/60.0)
                                                                                      ,np.exp(-a2*((t-rain_begin_time)
                                                                                                   /(60*60.0)-b2))),0))
                                st3 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt3 =Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h3 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a3 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b3= (st3/60/60.0+dt3/60/60.0) + np.log(h3)/a3
                                rain_inflow_2 = anuga.Inflow(domain,  polygon = rain_area2_polygon, 
                                                             rate = lambda t: max(min(h3/(dt3/60/60.0)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st3/60.0/60)
                                                                                      ,np.exp(-a3*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b3))),0))
                                st4 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt4 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h4 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a4 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b4= (st4/60/60.0+dt4/60/60.0) + np.log(h4)/a4
                                rain_inflow_3 = anuga.Inflow(domain,  polygon = rain_area3_polygon, 
                                                             rate = lambda t: max(min(h4/(dt4/60.0/60)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st4/60.0/60)
                                                                                      ,np.exp(-a4*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b4))),0))
                                st5 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt5 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h5 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a5 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b5 = (st5/60/60.0+dt5/60/60.0) + np.log(h5)/a5
                                rain_inflow_4 = anuga.Inflow(domain,  polygon = rain_area4_polygon, 
                                                             rate = lambda t: max(min(h5/(dt5/60.0/60)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st5/60.0/60)
                                                                                      ,np.exp(-a5*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b5))),0))
                                st6 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt6 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h6 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a6 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b6 = (st6/60.0/60+dt6/60.0/60) + np.log(h6)/a6
                                rain_inflow_5 = anuga.Inflow(domain, polygon = rain_area5_polygon, 
                                                             rate = lambda t: max(min(h6/(dt6/60.0/60)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st6/60.0/60)
                                                                                      ,np.exp(-a6*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b6))),0))
                                st7 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt7 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h7 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a7 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b7 = (st7/60.0/60.0+dt7/60.0/60.0) + np.log(h7)/a7
                                rain_inflow_6 = anuga.Inflow(domain, polygon = rain_area6_polygon, 
                                                             rate = lambda t: max(min(h7/(dt7/60.0/60)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st7/60.0/60)
                                                                                      ,np.exp(-a7*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b7))),0))
                                st8 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt8 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h8 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a8 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b8 = (st8/60.0/60.0+dt8/60.0/60.0) + np.log(h8)/a8
                                rain_inflow_7 = anuga.Inflow(domain,  polygon = rain_area7_polygon, 
                                                             rate = lambda t: max(min(h8/(dt8/60/60.0)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st8/60/60.0)
                                                                                      ,np.exp(-a8*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b8))),0))
                                st9 = St[i_st] + (St[i_st+1] - St[i_st]) * np.random.rand(1)
                                dt9 = Dt[i_dt] + (Dt[i_dt+1] - Dt[i_dt]) * np.random.rand(1)
                                h9 = H[ih] + (H[ih+1] - H[ih]) * np.random.rand(1)
                                a9 = A[i_a] + (A[i_a+1] - A[i_a]) * np.random.rand(1)
                                b9 = (st9/60.0/60.0+dt9/60.0/60.0) + np.log(h9)/a9
                                rain_inflow_8 = anuga.Inflow(domain, polygon = rain_area8_polygon, 
                                                             rate = lambda t: max(min(h9/(dt9/60.0/60)
                                                                                      *(((t-rain_begin_time)/(60.0*60))
                                                                                        - st9/60.0/60)
                                                                                      ,np.exp(-a9*((t-rain_begin_time)
                                                                                                   /(60.0*60)-b9))),0))
                                # simulation
                                # save all the inflows and rain data
                                t0 = time.time()
                                for t in domain.evolve(yieldstep=5*60, finaltime = 30*60):
                                    print domain.time
                                    #print domain.timestepping_statistics()
                                if allclose(t, 30*60):
                                    #print 'yes'
                                    #print length_force
                                    del domain.forcing_terms[0:length_fulfill_force]
                                    #print domain.forcing_terms
                                    domain.forcing_terms.append(rain_inflow_0)
                                    domain.forcing_terms.append(rain_inflow_1)
                                    domain.forcing_terms.append(rain_inflow_2)
                                    domain.forcing_terms.append(rain_inflow_3)
                                    domain.forcing_terms.append(rain_inflow_4)
                                    domain.forcing_terms.append(rain_inflow_5)
                                    domain.forcing_terms.append(rain_inflow_6)
                                    domain.forcing_terms.append(rain_inflow_7)
                                    domain.forcing_terms.append(rain_inflow_8)
                                for t in domain.evolve(yieldstep=5*60, finaltime=2*60*60, skip_initial_step=True):
                                    rain1 = 0
                                    rain2 = 0
                                    rain3 = 0
                                    rain4 = 0
                                    rain5 = 0
                                    rain6 = 0
                                    rain7 = 0
                                    rain8 = 0
                                    rain9 = 0
                                    inflow1 = in1 + 0.1*in1*sin(t/10.0)
                                    inflow2 = in2 + 0.1*in2*sin(t/10.0)
                                    inflow3 = in3 + 0.1*in3*sin(t/10.0)
                                    inflow4 = in4 + 0.1*in4*sin(t/10.0)
                                    inflow5 = in5 + 0.1*in5*sin(t/10.0)
                                    inflow10 = in10 + 0.1*in10*sin(t/10.0)
                                    outflow1 = out1 - 0.1*out1*sin(t/10.0)
                                    temp = [rain1, rain2, rain3, rain4, rain5, rain6, rain7, rain8, rain9, 
                                            inflow1,inflow2, inflow3, inflow4, inflow5, inflow10, outflow1]
                                    input_info.append(temp)
                                    if domain.time == 2*60*60.0:
                                        print domain.time
                                for t in domain.evolve(yieldstep=5*60, finaltime=14*60*60, skip_initial_step=True): 
                                    # save the inflow/rain at each step
                                    rain1 = max(min(h1/(dt1/60.0/60)*(((t-rain_begin_time)/(60*60.0)) - st1/60.0/60),
                                                    np.exp(-a1*((t-rain_begin_time)/(60*60.0)-b1))),0.0)
                                    rain2 = max(min(h2/(dt2/60/60.0)*(((t-rain_begin_time)/(60.0*60)) - st2/60/60.0),
                                                    np.exp(-a2*((t-rain_begin_time)/(60*60.0)-b2))),0)
                                    rain3 = max(min(h3/(dt3/60/60)*(((t-rain_begin_time)/(60.0*60)) - st3/60/60.0),
                                                    np.exp(-a3*((t-rain_begin_time)/(60*60.0)-b3))),0)
                                    rain4 = max(min(h4/(dt4/60/60)*(((t-rain_begin_time)/(60.0*60)) - st4/60/60.0),
                                                    np.exp(-a4*((t-rain_begin_time)/(60*60.0)-b4))),0)
                                    rain5 = max(min(h5/(dt5/60/60)*(((t-rain_begin_time)/(60.0*60)) - st5/60/60.0),
                                                    np.exp(-a5*((t-rain_begin_time)/(60*60.0)-b5))),0)
                                    rain6 = max(min(h6/(dt6/60/60)*(((t-rain_begin_time)/(60.0*60)) - st6/60/60.0),
                                                    np.exp(-a6*((t-rain_begin_time)/(60*60.0)-b6))),0)
                                    rain7 = max(min(h7/(dt7/60/60)*(((t-rain_begin_time)/(60.0*60)) - st7/60/60.0),
                                                    np.exp(-a7*((t-rain_begin_time)/(60*60.0)-b7))),0)
                                    rain8 = max(min(h8/(dt8/60/60)*(((t-rain_begin_time)/(60.0*60)) - st8/60/60.0),
                                                    np.exp(-a8*((t-rain_begin_time)/(60*60.0)-b8))),0)
                                    rain9 = max(min(h9/(dt9/60/60)*(((t-rain_begin_time)/(60.0*60)) - st9/60/60.0),
                                                    np.exp(-a9*((t-rain_begin_time)/(60*60.0)-b9))),0)
                                    inflow1 = in1 + 0.1*in1*sin(t/10.0)
                                    inflow2 = in2 + 0.1*in2*sin(t/10.0)
                                    inflow3 = in3 + 0.1*in3*sin(t/10.0)
                                    inflow4 = in4 + 0.1*in4*sin(t/10.0)
                                    inflow5 = in5 + 0.1*in5*sin(t/10.0)
                                    inflow10 = in10 + 0.1*in10*sin(t/10.0)
                                    outflow1 = out1 - 0.1*out1*sin(t/10.0)
                                    temp = [rain1, rain2, rain3, rain4, rain5, rain6, rain7, rain8, rain9, 
                                            inflow1,inflow2, inflow3, inflow4, inflow5, inflow10, outflow1]
                                    input_info.append(temp)
                                    
                                    #input_matrix(saving_file, case_name, int(t), rain1, rain2, rain3, 
                                    #             rain4, rain5, rain6, rain7, rain8, rain9,
                                    #             inflow1, inflow2, inflow3, inflow4, inflow5, inflow10, outflow1)
                                    if domain.time== 8*60*60.0:
                                        print 'Simulation Half way'
                                    #print domain.timestepping_statistics()
                                if mark == 0:
                                    ele, matrix_A = get_matrix_A(sww_fn, gauge_fn)
                                    mark = mark+1
                                
                                print 'Writing the input information'
                                input_info = np.asarray(input_info)
                                input_name = saving_file+'input'+case_name+'.csv'
#                                 np.savetxt(input_name, input_info, delimiter = ",")
                                input_info = []
                                # interpolate all the points and save them every 10 minutes
                                print 'Writing the interpolation result'
                                
                                # get the interpolation result
#                                 read_result(sww_fn, saving_file, matrix_A, ele, quantities, case_name)

Default false northing is 10000000.000000.
ANUGA does not correct for differences in False Northings.
Default units is m.
ANUGA does not correct for differences in units.
This is case 0_0_0_0_0_0_0_2
0.0
300.0
600.0
900.0
1200.0
1500.0
1800.0
7200.0
Simulation Half way
Writing the input information
Writing the interpolation result
This is case 0_0_0_0_0_1_0_2
0.0
300.0
600.0
900.0
1200.0
1500.0
1800.0


KeyboardInterrupt: 